In [2]:
# %%
import qutip as qt
import cvxpy as cp
import numpy as np

# ---------------------------
# Goal: map |+><+|  ->  |0><0|
# ---------------------------
ket0  = qt.basis(2, 0)
plus  = (qt.basis(2, 0) + qt.basis(2, 1)).unit()

rho_np   = qt.ket2dm(plus).full()   # input
sigma_np = qt.ket2dm(ket0).full()   # target

# ---------------------------
# Variables
# ---------------------------
J = cp.Variable((4, 4), complex=True)   # Choi matrix on Y⊗X (output ⊗ input)
W = cp.Variable((2, 2), complex=True)   # fidelity witness
t = cp.Variable()               # lower bound for sqrtF; we maximize t, and report F=t^2

# ---------------------------
# Helper: slice J into 2x2 blocks J_ij over input indices (i=row, j=col)
# In Y⊗X ordering, each block J_ij is a 2x2 matrix over the OUTPUT space Y.
# ---------------------------
def J_block(i, j):
    r = slice(2*i, 2*(i+1))
    c = slice(2*j, 2*(j+1))
    return J[r, c]

# ---------------------------
# Channel action via block formula:
#   Φ(ρ) = sum_{i,j} ρ_{j,i} * J_{i,j},  where J_{i,j} = Φ(|i><j|)
# ---------------------------
Phi_rho = 0
for i in range(2):
    for j in range(2):
        Phi_rho += rho_np[j, i] * J_block(i, j)

# ---------------------------
# CPTP constraints:
# 1) J ⪰ 0
# 2) Trace-preserving: Tr_Y J = I_X  =>  trace(J_ij) = δ_ij
# ---------------------------
constraints = [J >> 0]
delta = np.eye(2)
for i in range(2):
    for j in range(2):
        constraints.append(cp.trace(J_block(i, j)) == delta[i, j])

# (Optional stabilizers)
constraints += [Phi_rho >> 0, cp.trace(Phi_rho) == 1]

# ---------------------------
# Fidelity SDP:
#   [ Φ(ρ)   W
#     W†     σ ]  ⪰ 0,   Re Tr(W) ≥ t
# Maximize t  (so the optimal fidelity is F = t^2)
# ---------------------------
block = cp.bmat([[Phi_rho, W],
                 [W.H,     sigma_np]])
constraints += [block >> 0, cp.real(cp.trace(W)) >= t]

objective = cp.Maximize(t)
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS, verbose=True)

sqrtF = float(prob.value)
F = sqrtF**2
print(f"Optimal sqrt(F) = {sqrtF:.6f}")
print(f"Optimal F       = {F:.6f}")
print("Φ(ρ) ≈\n", Phi_rho.value)
print("Choi J(Φ) ≈\n", J.value)

# ---------------------------
# Diagnostics: Kraus extraction + quick unitary check
# ---------------------------
# Eigen-decomp of J gives Kraus via column-major reshape (vec convention)
evals, evecs = np.linalg.eigh(J.value)
kraus = []
for lam, v in zip(evals, evecs.T):
    if lam > 1e-10:
        K = (np.sqrt(lam) * v.reshape(2, 2, order='F'))
        kraus.append(K)

# Check TP: sum K†K ≈ I
TP_check = np.allclose(sum(K.conj().T @ K for K in kraus), np.eye(2), atol=1e-6)
print("TP check from Kraus (Σ K†K ≈ I):", TP_check)

# If it’s close to a unitary, there should be (numerically) one dominant Kraus
if len(kraus) == 1:
    U = kraus[0]
    # up to a global phase, U should be close to Hadamard for this task
    H = np.array([[1, 1], [1, -1]])/np.sqrt(2)
    # remove a global phase and compare
    phase = np.linalg.det(U)**(-0.5)
    U_phase_fixed = phase * U
    print("‖U - H‖_F =", np.linalg.norm(U_phase_fixed - H))
else:
    print(f"{len(kraus)} Kraus operators found (channel may be non-unitary).")


(CVXPY) Nov 03 07:40:21 PM: Your problem has 21 variables, 42 constraints, and 0 parameters.
(CVXPY) Nov 03 07:40:21 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 03 07:40:21 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 03 07:40:21 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 03 07:40:21 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Nov 03 07:40:21 PM: Compiling problem (target solver=SCS).
(CVXPY) Nov 03 07:40:21 PM: Reduction chain: Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCS
(CVXPY) Nov 03 07:40:21 PM: Applying reduction Complex2Real
(CVXPY) Nov 03 07:40:21 PM: Applying reduction FlipObjective
(CVXPY) Nov 03 07:40:21 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 03 07:40:21 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 03 07:40:21 PM: App

                                     CVXPY                                     
                                     v1.7.2                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
------------------------------------------------------------------
	       SCS v3.2.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 41, constraints m: 93
cones: 	  z: primal zero / dual free vars: 10
	  l: linear vars: 1
	  s: psd vars: 82,